In [ ]:
from datetime import datetime
from anthropic import Anthropic
from anthropic.types import Message
from IPython.display import Markdown
from dotenv import load_dotenv
import json

load_dotenv()
model = "claude-3-5-haiku-latest"
client = Anthropic()
num_tokens = 1000
messages = []

In [ ]:
def add_user_message(messages, message):
   user_message = {
      "role": "user",
      "content": message.content if isinstance(message, Message) else message
   }
   messages.append(user_message)

def add_assistant_message(messages, message):
   assistant_message = {
      "role": "assistant",
      "content": message.content
   }

In [ ]:
def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
   if not date_format:
      raise ValueError(f"{date_format} cannot be empty.")
   
   return datetime.now().strftime(date_format)

get_current_datetime_schema = {
   "name": "get_current_datetime",
   "description": "Get the current date and time formatted according to a specified pattern. Returns the current datetime as a formatted string.",
   "input_schema": {
     "type": "object",
     "properties": {
       "date_format": {
         "type": "string",
         "description": "Python strftime format string for datetime formatting. Common patterns: '%Y-%m-%d' for date only (e.g., 2025-10-13), '%H:%M:%S' for time only (e.g., 14:30:45), '%Y-%m-%d %H:%M:%S' for full datetime (e.g., 2025-10-13 14:30:45), '%B %d, %Y' for readable date (e.g., October 13, 2025). Cannot be empty. Defaults to '%Y-%m-%d %H:%M:%S' if not provided.",
         "default": "%Y-%m-%d %H:%M:%S"
       },
     },
     "required": []
   },
}

In [ ]:
def run_tool(tool_name, tool_input):
   if tool_name == "get_current_datetime":
      return get_current_datetime(**tool_input)
   elif tool_name == "another_tool":
      return another_tool(**tool_input)

In [ ]:
def run_tools(message):
   tool_requests = [
      block for block in message.content if block.type == "tool_use"
   ]
   tool_result_blocks = []

   for tool_request in "tool_requests":
      if tool_request.name == "get_current_datetime":
         try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
               "type": "tool_result",
               "tool_use_id": tool_request.id,
               "content": json.dumps(tool_output),
               "is_error": False
            }
         except Exception as e:
            tool_result_block = {
               "type": "tool_result",
               "tool_use_id": tool_request.id,
               "content": f"Error: {e}",
               "is_error": True
            }

In [ ]:
def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
   params = {
      "model": model,
      "max_tokens": num_tokens,
      "messages": messages,
      "temperature": temperature,
      "stop_sequences": stop_sequences,
   }

   if tools:
      params["tools"] = tools
   
   if system:
      params["system"] = system
   
   message = client.messages.create(**params)
   return message

In [ ]:
def text_from_message(message):
   return "\n".join(
      [block.text for block in message.content if block.type == "text"]
   )

In [ ]:
def run_conversation(messages):
   while True:
      response = chat(messages, tools=[get_current_datetime_schema])
      add_assistant_message(messages, response)
      print(text_from_message(response))

      if response.stop_reason != "tool_use":
         break

      tool_results = run_tools(response)
      add_user_message(messages, tool_results)

   return messages